In [1]:
import pandas as pd
import numpy as np
import json

In [2]:
test_df=pd.read_csv('./output/train_statement_section.csv')
test_df

,statement,section,label
0,All the primary trial participants do not rece...,"[""INTERVENTION 1:"", ""Diagnostic (FLT PET)"", ""P...",Contradiction
1,"Patients with Platelet count over 100,000/mm¬¨...","[""DISEASE CHARACTERISTICS:"", ""Histologically o...",Contradiction
2,Heart-related adverse events were recorded in ...,"[""Adverse Events 1:"", ""Total: 5/32 (15.63%)"", ...",Entailment
3,Adult Patients with histologic confirmation of...,"[""Inclusion Criteria:"", ""Patients with histolo...",Contradiction
4,Laser Therapy is in each cohort of the primary...,"[""INTERVENTION 1:"", ""Laser Therapy Alone"", ""th...",Contradiction
...,...,...,...
1695,"Adequate blood, kidney, and hepatic function a...","[""Inclusion Criteria:"", ""Postmenopausal women,...",Entailment
1696,The Ridaforolimus + Dalotuzumab + Exemestane g...,"[""Outcome Measurement:"", ""1. Progression-free ...",Contradiction
1697,The only difference between the interventions ...,"[""INTERVENTION 1:"", ""Prone"", ""Prone position"",...",Entailment
1698,Patients must have a white blood cell count ab...,"[""DISEASE CHARACTERISTICS:"", ""Histologically c...",Entailment


In [3]:
hypothesis_lst=test_df['statement'].values.tolist()
len(hypothesis_lst)

1700

In [4]:
evidence_lst=test_df['section'].apply(lambda l:' '.join(json.loads(l))).values.tolist()
len(evidence_lst)

1700

In [5]:
label2id={"Contradiction":0,"Entailment":1}
label_lst=test_df['label'].apply(lambda x:label2id[x]).values.tolist()
len(label_lst)

1700

In [6]:
import random
import math
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F

from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [7]:
# text_tok=AutoTokenizer.from_pretrained('bert-base-uncased')
# text_clf=AutoModelForSequenceClassification.from_pretrained('bert-base-uncased',num_labels=2)
#roberta-base
# text_tok=AutoTokenizer.from_pretrained('roberta-base')
# text_clf=AutoModelForSequenceClassification.from_pretrained('roberta-base',num_labels=2)
#bio-bert
# text_tok=AutoTokenizer.from_pretrained('dmis-lab/biobert-v1.1')
# text_clf=AutoModelForSequenceClassification.from_pretrained('dmis-lab/biobert-v1.1',num_labels=2)
#gpt2
text_tok=AutoTokenizer.from_pretrained('gpt2')
text_tok.pad_token = text_tok.eos_token
text_clf=AutoModelForSequenceClassification.from_pretrained('gpt2',num_labels=2,pad_token_id=text_tok.eos_token_id)

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
class InputSequence:
    
    def __init__(self,tok,l_text,l_text2,l_label,batch_size=64,gpu=True):
        
        self.data_len=len(l_text)
        self.data_idx=[i for i in range(self.data_len)]
        self.texts=tok(l_text,l_text2,padding=True, truncation=True, max_length=512, return_tensors='pt')
        self.l_label=np.array(l_label)
        print('tokenize done')
        
        self.batch_size=batch_size
        self.gpu=gpu
        
    def on_epoch_end(self):
        random.shuffle(self.data_idx)
        
    def __getitem__(self,i):
        start=i*self.batch_size
        batch_idx=self.data_idx[start:min(start+self.batch_size,self.data_len)]
        
        return_texts=dict([(k,self.texts[k][batch_idx]) for k in self.texts])
        return_labels=torch.from_numpy(
            self.l_label[batch_idx].astype(np.int64)
        )
        
        if self.gpu:
            return_texts=dict([(k,return_texts[k].cuda()) for k in return_texts])
            return_labels=return_labels.cuda()
        
        return return_texts,return_labels
    
    def __len__(self):
        return math.ceil(1.0*self.data_len/self.batch_size)
    

In [9]:
testing_data=InputSequence(text_tok,hypothesis_lst,evidence_lst,label_lst,gpu=True)

tokenize done


In [10]:
scores=[]
# model_names=['bert-base-uncased']+[
#     './output/clf_models/bert-base-uncased_epoch_{}.pt'.format(format(epoch,'05d'))
#     for epoch in range(10)
# ]
#roberta-base
# model_names=['roberta-base']+[
#     './output/clf_models/roberta-base_epoch_{}.pt'.format(format(epoch,'05d'))
#     for epoch in range(10)
# ]
#biobert
# model_names=['dmis-lab/biobert-v1.1']+[
#     './output/clf_models/biobert-v1.1_epoch_{}.pt'.format(format(epoch,'05d'))
#     for epoch in range(10)
# ]
#gpt2
model_names=['gpt2']+[
    './output/clf_models/gpt2_epoch_{}.pt'.format(format(epoch,'05d'))
    for epoch in range(10)
]
for model_name in model_names:
    scores.append([])
    clf=AutoModelForSequenceClassification.from_pretrained(model_name,num_labels=2,pad_token_id=text_tok.eos_token_id).cuda()
    with torch.no_grad():
        for batch in range(len(testing_data)):
            batch_texts,batch_labels=testing_data[batch]
            scores[-1].append(F.softmax(clf(**batch_texts).logits,dim=1).detach().cpu().numpy())
            print('model:',model_name,'batch:',batch,end='\r')
    scores[-1]=np.concatenate(scores[-1],axis=0)
    clf.cpu()

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
from sklearn.metrics import average_precision_score,f1_score,precision_score,recall_score,accuracy_score

y_true=label_lst
results=[]
for epoch in range(len(scores)):
    y_prob=scores[epoch][:,1]
    y_pred=[1 if a>0.5 else 0 for a in y_prob]
    results.append([
        'pretrained' if epoch==0 else epoch,
        average_precision_score(y_true,y_prob),
        f1_score(y_true,y_pred),
        precision_score(y_true,y_pred),
        recall_score(y_true,y_pred),
        accuracy_score(y_true,y_pred)
    ])

import pandas as pd

pd.DataFrame(results,columns=['epoch','AVG_PREC','F1','PREC','REC','ACC'])

,epoch,AVG_PREC,F1,PREC,REC,ACC
0,pretrained,0.506511,0.286198,0.510574,0.198824,0.504118
1,1,0.510473,0.566316,0.512381,0.632941,0.515294
2,2,0.528301,0.603269,0.520958,0.716471,0.528824
3,3,0.544654,0.503686,0.526992,0.482353,0.524706
4,4,0.562857,0.613783,0.530420,0.728235,0.541765
5,5,0.582067,0.625490,0.536134,0.750588,0.550588
6,6,0.609362,0.524778,0.570442,0.485882,0.560000
7,7,0.636676,0.484976,0.597246,0.408235,0.566471
8,8,0.662759,0.676726,0.553478,0.870588,0.584118
9,9,0.672117,0.645943,0.584924,0.721176,0.604706
